# nornir_rich
nornir_rich plugin is a combination of a processor to get additional detail for results and related functions.  By default all results are output to stdout without requiring and print_result statements.

The processor constructor has some options related to keeping track of timing, screen width and whether or not to utilize progress bars.  The example below uses all defaults.

In [1]:
from nornir import InitNornir
from nornir_napalm.plugins.tasks import napalm_get

nr = InitNornir(
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "inventory/hosts.yaml",
            "group_file": "inventory/groups.yaml",
            "defaults_file": "inventory/defaults.yaml",
        }
    },
)

After creating the standard Nornir object, a RichResults processor object is instantiated and added to Nornir.  The width of 67 in the example was only used as this notebook is going into read the docs via sphinx and without it the output required a scroll bar.  Normally leaving this at defaults will just use up max length of the terminal.

In [2]:
from nornir_rich.plugins.functions import RichResults
rr = RichResults()

A basic password transform is done on the inventory using dotenv applied environment variables.  This is just for test environment setup.

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

def passwd_from_env(task):
    task.host.password = os.getenv(task.host.password)

rr.print(nr.run(passwd_from_env))

By default, write_inventory function won't list passwords without the passwords=True argument.

In [4]:
rr.inventory(nr)

In [10]:
rr.print(nr.run(task=napalm_get, getters=['interfaces'], name='run1'))

The write_summary method will provide a summary of the run.

In [11]:
rr.summary()

Skip messages are displayed by default for skipped hosts when a task is run from nr with skipped hosts present.

In [12]:
nr.data.reset_failed_hosts()
from nornir.core.task import Result

def return_changed(task):
    return Result(task.host, changed=True)

def return_failed(task):
    return Result(host=task.host, failed=True)

rr.print(nr.run(return_changed))
rr.print(nr.run(return_failed))
rr.print(nr.run(return_changed))

In [13]:
rr.summary()

Output formats are mostly handled ok and can either be highlighted with rich or left as uncolored.  By default it is uncolored as the highlighting isn't exactly pleasing to my eyes minus the exceptions.

In [14]:
def return_diff(task):
    diff = open('inventory/test.diff', 'r').read()
    return Result(host=task.host, diff=diff, changed=True)

def return_json(task):
    json = open('inventory/test.json', 'r').read()
    return Result(host=task.host, stdout=json)

def return_xml(task):
    xml = open('inventory/test.xml', 'r').read()
    return Result(host=task.host, result=xml)

def grouped(task):
    task.run(return_diff)
    task.run(return_json)
    task.run(return_xml)

nr.data.reset_failed_hosts()
rr.print(nr.run(grouped))

TypeError: _print_result() missing 3 required positional arguments: 'vars', 'failed', and 'severity_level'

Or highlighted...